<a href="https://colab.research.google.com/github/sonahyoung/class2022Spring/blob/%EA%B8%B0%EB%A7%90%EB%B2%94%EC%9C%84/tensorflow_hub.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### [Tensorflow Hub](https://www.tensorflow.org/hub)

a repository of trained machine learning models

fine-tuning
입력 데이터 -> AI 행렬 -> 출력 벡터
paired된 데이터가 많이 있을때, 입력부분에 그게 들어오면 답을 맞출 수 있도록 행렬이 훈련된다. 

pre-trained된 모델이 있을때를 가정하자. 그리고 paired된 데이터를 기반으로 해서 모델이 만들어졌다고 치자. 원래는 코끼리 그림을 입력으로 출력으로는 코끼리라는 text를 놔뒀을때, 그걸 fine-tuning이라고 함.

pre-trained는 기존의 어떤 모델을 어떤 사람이 기존에 있는 데이터로 훈련시킨 것
fine-tuning은 내가 나의 데이터로 그 모델을 조금 확장시켰다.

TensorFlow hub is a repository of (pre-)trained machine learning model임.. -> 근데 fine-tuing을 해도 되고 deployable(바로 써도 된다)

### image classification (e.g. [inception_v3](https://tfhub.dev/google/imagenet/inception_v3/classification/5))

In [2]:
import tensorflow.compat.v2 as tf
import tensorflow_hub as hub

In [3]:
model = hub.KerasLayer("https://tfhub.dev/google/imagenet/inception_v3/classification/5")   #사이트는 inception_v3 classification 들어가서 copy url을 함.

In [7]:
from PIL import Image
import requests
url = 'http://images.cocodataset.org/val2017/000000039769.jpg'
image = Image.open(requests.get(url, stream=True).raw)  #이미지를 open을 했음   #image가 PIL의 형태의 데이터로 그냥 되어있음.

In [9]:
import numpy as np   #이미지를 바로 쓸 수 있는게 아니라 array 형태로 만들어줘야함.
image = np.array(image)   #여기 image는 (480, 640, 3)

In [ ]:
image[:, :, 0]   #숫자가 0~256으로, rgb에 대한 강도가 되어있을거임. 

In [12]:
# reshape into shape [batch_size, height, width, num_channels]   #image가 원래는 3차원의 array인데, 4차원의 array로 바꾸어줘야함. 모델 자체가 4차원을 받기 때문에
image = tf.reshape(image, [1, image.shape[0], image.shape[1], image.shape[2]])   #3차원의 array를 4차원의 array로 바꾸어주는 과정
# 여기서 image는 (1, 480, 640, 3) -> 그림이 2장 있으면 맨 앞 숫자가 2로 바뀌고 100장 있으면 100으로 바뀜.

In [14]:
# Use `convert_image_dtype` to convert to floats in the [0,1] range.   #우리가 쓰려고 하는 모델 자체는 [0, 1]로 되어야함.
image = tf.image.convert_image_dtype(image, tf.float32)
# 0~255 정수의 숫자가 아닌 0~1까지의 range로 바꿔주는거임. 모델이 그렇게 되어있으니까.

In [ ]:
image[0, :, :, 0]

In [20]:
logits = model(image)   #모델을 쓰는 곳은 여기임. 모델 안에 행렬이 엄청 들어있다.   #logits 값이 (1, 1001)로 1000개가 들어와있음.
logits

<tf.Tensor: shape=(1, 1001), dtype=float32, numpy=
array([[ 0.13278216, -0.98323435,  0.4061772 , ..., -0.12934873,
         0.38595918,  0.4459334 ]], dtype=float32)>

In [21]:
np.argmax(logits)  #logits값들 중에 어떤 것이 제일 높은 값인가, 1000개중에 어떤 것이 제일 높은 값인가가 argmax라는 함수임.
#실행되어서 나온 282는 282번째라는 거임.

282

> https://storage.googleapis.com/download.tensorflow.org/data/ImageNetLabels.txt

282번째 숫자가 의미하는 바를 아려면 여기 들어가면 됨.

inception_v3라는 모델이 훈련했던 게 있을거임. 그 데이터들이 어떻게 만들어졌냐면? 그림이 있고 강아지, 꽃과 같이 훈련을 시켰을 거임. 강아지, 꽃과 같은 label이 1000개가 있을거임. 

-> 0부터이니까 283번째를 보면 tabby라고 적혀있음.